In [16]:
import numpy as np
import pandas as pd
import config_final
import requests
from bs4 import BeautifulSoup as BS


In [17]:
config_final.user

'munzma'

# Webscrape URL List for Bill Summaries

In [18]:
raw = pd.read_csv('bills93-114.csv', sep=";", encoding='latin-1')

df = raw[['BillID','URL']]
# df = df.set_index('BillID')
df.head()

,BillID,URL
0,93-HCONRES-1,https://www.congress.gov/bill/93th-congress/ho...
1,93-HCONRES-2,https://www.congress.gov/bill/93th-congress/ho...
2,93-HCONRES-3,https://www.congress.gov/bill/93th-congress/ho...
3,93-HCONRES-4,https://www.congress.gov/bill/93th-congress/ho...
4,93-HCONRES-5,https://www.congress.gov/bill/93th-congress/ho...


In [19]:
df['URL'] = df['URL'].astype('str') 

/Users/melissamunz/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [20]:
# df = df.drop([0])
# df = df.drop([1])
# df.head()

In [21]:
df.iloc[0]

BillID                                         93-HCONRES-1
URL       https://www.congress.gov/bill/93th-congress/ho...
Name: 0, dtype: object

In [22]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 294524 entries, 0 to 294523
Data columns (total 2 columns):
BillID    294524 non-null object
URL       294524 non-null object
dtypes: object(2)
memory usage: 4.5+ MB


In [23]:
# df.iloc[0][1]

## Create Scraper

In [24]:
def scraper(url):
       
    response = requests.get(url)
    soup = BS(response.content, "html.parser")
    soup.prettify() 
    
    try:
        return soup.find('div', id="bill-summary").find_all('p')[1].get_text(strip=True)
        
    except:
        print(url)          
    

In [25]:
df1 = df[0:10]
df1

,BillID,URL
0,93-HCONRES-1,https://www.congress.gov/bill/93th-congress/ho...
1,93-HCONRES-2,https://www.congress.gov/bill/93th-congress/ho...
2,93-HCONRES-3,https://www.congress.gov/bill/93th-congress/ho...
3,93-HCONRES-4,https://www.congress.gov/bill/93th-congress/ho...
4,93-HCONRES-5,https://www.congress.gov/bill/93th-congress/ho...
5,93-HCONRES-6,https://www.congress.gov/bill/93th-congress/ho...
6,93-HCONRES-7,https://www.congress.gov/bill/93th-congress/ho...
7,93-HCONRES-8,https://www.congress.gov/bill/93th-congress/ho...
8,93-HCONRES-9,https://www.congress.gov/bill/93th-congress/ho...
9,93-HCONRES-10,https://www.congress.gov/bill/93th-congress/ho...


In [26]:
df1['Summary'] = df1.apply (lambda row: scraper(row['URL']), axis=1)

https://www.congress.gov/bill/93th-congress/house-concurrent-resolution/1


/Users/melissamunz/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [27]:
df1

,BillID,URL,Summary
0,93-HCONRES-1,https://www.congress.gov/bill/93th-congress/ho...,None
1,93-HCONRES-2,https://www.congress.gov/bill/93th-congress/ho...,Makes it the sense of the Congress that the po...
2,93-HCONRES-3,https://www.congress.gov/bill/93th-congress/ho...,Establishes a Joint Congressional Committee on...
3,93-HCONRES-4,https://www.congress.gov/bill/93th-congress/ho...,Makes it the sense of the Congress that the Pr...
4,93-HCONRES-5,https://www.congress.gov/bill/93th-congress/ho...,Makes it the sense of the Congress that: (1) ...
5,93-HCONRES-6,https://www.congress.gov/bill/93th-congress/ho...,Makes it the sense of Congress that it shall b...
6,93-HCONRES-7,https://www.congress.gov/bill/93th-congress/ho...,Makes it the sense of the Congress that the Un...
7,93-HCONRES-8,https://www.congress.gov/bill/93th-congress/ho...,Expresses the sense of the Congress that the S...
8,93-HCONRES-9,https://www.congress.gov/bill/93th-congress/ho...,Expresses the sense of the Congress that all A...
9,93-HCONRES-10,https://www.congress.gov/bill/93th-congress/ho...,Makes it the sense of the Congress that the re...


In [ ]:
df['Summary'] = df.apply (lambda row: scraper(row['URL']), axis=1)

https://www.congress.gov/bill/93th-congress/house-concurrent-resolution/1
https://www.congress.gov/bill/93th-congress/house-concurrent-resolution/41
https://www.congress.gov/bill/93th-congress/house-concurrent-resolution/42
https://www.congress.gov/bill/93th-congress/house-concurrent-resolution/43
https://www.congress.gov/bill/93th-congress/house-concurrent-resolution/78
https://www.congress.gov/bill/93th-congress/house-concurrent-resolution/105
https://www.congress.gov/bill/93th-congress/house-concurrent-resolution/132


In [ ]:
df.head()

## Enter into Database

In [ ]:
from sodapy import Socrata
import sqlalchemy as db

In [ ]:
def query(q):
    try:
        return pd.read_sql_query(q, engine)
    # Pass when no data is returned    
    except ResourceClosedError:
        pass

def query_list(col, table, distinct = True):
    elts = ['SELECT',
            'DISTINCT' if distinct else '',
            col,
            'FROM',
            table]
    query_str = ' '.join(elts)
    df = query(query_str)
    l = df.iloc[:,0].tolist()
    return l

In [ ]:
df.iloc[0][1]

In [ ]:
len(df.iloc[3][1])

In [ ]:
#instantiate the connection and table

engine = db.create_engine(f'mysql+mysqlconnector://{config_final.user}:{config_final.password}@{config_final.host}/con_bills')

connection = engine.connect()
metadata=db.MetaData()

**Summaries Table**

In [ ]:
summaries = db.Table('summaries', metadata, 
    db.Column('Summary', db.String(500)),               
    db.Column('BillID', db.String(40), primary_key=True)
)

In [ ]:
metadata.create_all(engine)

In [ ]:
query = db.insert(summaries)

In [ ]:
connection.execute(query, url_call(df))